In [3]:
import pandas as pd
import numpy as np

Usamos base de datos de kaggle que utiliza el prefijo del codigo postal para normalizar los datos.

In [4]:
df = pd.read_csv(r'C:\Users\marco\Desktop\ProyectoGrupalH\data\geolocation_dataset_brazil.csv')

In [5]:
#Dropeamos duplicados y renombramos columnas para unificar descripcion de mismo contenido de columna
df.drop_duplicates(inplace=True)
df = df.rename(columns={'geolocation_zip_code_prefix': 'zip_code', 'geolocation_lat':'geolatitud','geolocation_lng':'geolongitud', 'geolocation_city':'city', 'geolocation_state':'state'})

In [6]:
#Utilizamos este json para normalizar el campo state
dfj = pd.read_json(r'C:\Users\marco\Desktop\ProyectoGrupalH\Json\estados.json')

In [7]:
#Mergeamos ambas tablas
df = df.merge(dfj[['UF-nome', 'UF-sigla']], right_on='UF-sigla', left_on='state')
#Dropeamos columna 'city_old' que hace referencia a un valor no normalizado
df.drop(columns=['geolocation_city_old'], inplace=True)
#Renombramos columnas para unificar descripcion de mismo contenido de columna
df.rename(columns={'UF-nome':'state_name'}, inplace=True)
#Normalizamos mayúsculas
df['city'] = df['city'].str.title()
#Eliminamos duplicados por 'zip_code' 
df = df.drop_duplicates('zip_code')
#Reseteamos indice
df = df.reset_index(drop=True)
#Finalmente, nos quedamos con las columnas que nos servirán a futuro.
df = df[['zip_code', 'geolatitud', 'geolongitud', 'city', 'state', 'state_name']]

In [8]:
df.to_parquet(r'C:\Users\marco\Desktop\ProyectoGrupalH\dataparquet\geolocation_cleanP')